In [1]:
# this is a refactor of the classification script to try to keep things tidied
# TODO: load the alien_speech_model


In [2]:
import numpy as np
import tensorflow as tf

from tensorflow.keras import models
from pydub import AudioSegment

loaded_model = tf.keras.models.load_model('alien.h5')


INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [3]:
char_classes = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '<', '_', 'B', 'C', 'H', 'L', 'M', 'N', 'R', 'T', 'U', '{', '}', 'A', 'Z', 'V', ':', 'D', '!', '.', 'J', 'X', 'G', '>', 'P', 'O', 'W', 'E', '&', 'Q', 'S', 'F', 'K', 'I', 'D', 'Y']

In [4]:
# get only the waveform from the wav file
def decode_audio(audio_binary):
  audio, _ = tf.audio.decode_wav(audio_binary)
  return tf.squeeze(audio, axis=-1)

# read the wav file and return the waveform
def get_waveform(file_path):
  audio_binary = tf.io.read_file(file_path)
  waveform = decode_audio(audio_binary)
  return waveform

# builds a spectrogram out of the waveform (using a short-time fourier transform)
# can handle at most 16000 samples = 1 second
def get_spectrogram(waveform):
  zero_padding = tf.zeros([16000] - tf.shape(waveform), dtype=tf.float32)

  waveform = tf.cast(waveform, tf.float32)
  equal_length = tf.concat([waveform, zero_padding], 0)
  spectrogram = tf.signal.stft(
      equal_length, frame_length=255, frame_step=128)
      
  spectrogram = tf.abs(spectrogram)

  return spectrogram

# adds a dimension to the spectrogram data structure
def get_spectrogram_audio(audio):
  spectrogram = get_spectrogram(audio)
  spectrogram = tf.expand_dims(spectrogram, -1)
  return spectrogram

# extracts features from the audio file
def preprocess_dataset(files):
  files_ds = tf.data.Dataset.from_tensor_slices(files)
  output_ds = files_ds.map(get_waveform, num_parallel_calls=tf.data.AUTOTUNE)
  output_ds = output_ds.map(
      get_spectrogram_audio,  num_parallel_calls=tf.data.AUTOTUNE)
  return output_ds

In [5]:
def classify_phoneme(filename):
    sample_ds = preprocess_dataset([str(filename)])

    for spectrogram in sample_ds.batch(1):
        # attempts to match the phoneme to a character class
        # alien_speech_model: the trained machine learning model that will try to classify the phoneme
        prediction = np.array(alien_speech_model(spectrogram)[0])
        class_index = np.argmax(prediction)
        return char_classes[np.argmax(prediction)]

In [6]:
# currently only classifying phonemes separately
# TODO: make it work with a whole sentence

char_class = classify_phoneme('alien_phoneme.wav')

print(char_class)

InvalidArgumentError: Dimension -1165104 must be >= 0
	 [[{{node zeros}}]] [Op:IteratorGetNext]